# Coordinate-Style Assignments with pandas.MultiIndex
This notebook demonstrates how to use pandas MultiIndex to simulate coordinate-style data management. We'll explore the creation of a MultiIndex DataFrame, assigning values to specific coordinates, batch assignments, adding new coordinates, querying, and defining a helper function to simplify the process.

In [15]:
# Importing pandas
import pandas as pd

# Creating a pandas DataFrame with a MultiIndex using `from_tuples`
index = pd.MultiIndex.from_tuples(
    [
        (0, 40.7128, -74.0060),  # New York
        (1, 34.0522, -118.2437),  # Los Angeles
        (2, 37.7749, -122.4194),  # San Francisco
    ],
    names=["time", "latitude", "longitude"]
)

df = pd.DataFrame({"value": [1.0, 2.0, 3.0]}, index=index)
print("Structure of the DataFrame:")
print(df)

Structure of the DataFrame:
                         value
time latitude longitude       
0    40.7128  -74.0060     1.0
1    34.0522  -118.2437    2.0
2    37.7749  -122.4194    3.0


In [10]:
# Assigning specific values to individual coordinates using `df.loc`

# Assign a new value to the coordinate (time=0, latitude=40.7128, longitude=-74.0060)
df.loc[(0, 40.7128, -74.0060), "value"] = 10.0

# Print updated DataFrame
print("DataFrame after assigning a value to a specific coordinate:")
print(df)

DataFrame after assigning a value to a specific coordinate:
                         value
time latitude longitude       
0    40.7128  -74.0060    10.0
1    34.0522  -118.2437    2.0
2    37.7749  -122.4194    3.0


In [11]:
# Batch assigning values to multiple coordinates using a dictionary of coordinate-value pairs

# Dictionary containing coordinates and their new values
coordinate_value_pairs = {
    (1, 34.0522, -118.2437): 20.0,  # Los Angeles
    (2, 37.7749, -122.4194): 30.0,  # San Francisco
}

# Updating values in the DataFrame
for coords, value in coordinate_value_pairs.items():
    df.loc[coords, "value"] = value

print("DataFrame after batch assignment of values:")
print(df)

DataFrame after batch assignment of values:
                         value
time latitude longitude       
0    40.7128  -74.0060    10.0
1    34.0522  -118.2437   20.0
2    37.7749  -122.4194   30.0


# Querying values by specific coordinates and filtering rows by MultiIndex levels

In [13]:
# Querying values by specific coordinates and filtering rows by MultiIndex levels

# Query by specific coordinate
coordinate = (1, 34.0522, -118.2437)  # Los Angeles
value = df.loc[coordinate, "value"]
print(f"Value for coordinate {coordinate}: {value}")

# Filtering rows by a specific latitude
filtered_df = df[df.index.get_level_values("latitude") == 40.7128]
print("Rows filtered by latitude=40.7128:")
print(filtered_df)

Value for coordinate (1, 34.0522, -118.2437): 20.0
Rows filtered by latitude=40.7128:
                         value
time latitude longitude       
0    40.7128  -74.006     10.0


In [14]:
# Defining a helper function for setting or adding coordinate values

def set_value(df, coordinates, value):
    """
    Sets the value for given coordinates. If the coordinates don't exist,
    they are added to the DataFrame.

    Parameters:
        df (pd.DataFrame): Target DataFrame with MultiIndex.
        coordinates (tuple): Coordinates to set.
        value (float): Value to assign.
    """
    df.loc[coordinates, "value"] = value

# Demonstrating the use of the helper function
set_value(df, (5, 35.6895, 139.6917), 60.0)  # Tokyo
set_value(df, (1, 34.0522, -118.2437), 25.0)  # Update Los Angeles

print("DataFrame after using the `set_value` function:")
print(df)

DataFrame after using the `set_value` function:
                         value
time latitude longitude       
0    40.7128  -74.0060    10.0
1    34.0522  -118.2437   25.0
2    37.7749  -122.4194   30.0
3    51.5074  -0.1278     40.0
4    48.8566   2.3522     50.0
5    35.6895   139.6917   60.0


## RDB-like access to Pandas

In [23]:
import pandas as pd

# Define data
data = [
    (0, 0, 30, 0.2),
    (1, 1, 35, 0.25),
    (2, 2, 40, 0.3),
]

# Load into DataFrame
df = pd.DataFrame(data, columns=["x", "y", "temperature", "humidity"])

print(df)

# Set keys for indexing
df = df.set_index(["x", "y"])

# Query/filter
result = df[df["temperature"] > 30]
print(result)

# Group and aggregate
avg_temp = df["temperature"].mean()
print(f"Average temperature: {avg_temp}")


   x  y  temperature  humidity
0  0  0           30      0.20
1  1  1           35      0.25
2  2  2           40      0.30
     temperature  humidity
x y                       
1 1           35      0.25
2 2           40      0.30
Average temperature: 35.0


## Duck DB to query Pandas datasets

In [22]:
import duckdb
import pandas as pd

# Setup data
data = [
    (0, 0, 30, 0.2),
    (1, 1, 35, 0.25),
    (2, 2, 40, 0.3),
]
df = pd.DataFrame(data, columns=["x", "y", "temperature", "humidity"])

print(df)

df.set_index(["x", "y"])

# Query DataFrame using DuckDB
query = """
SELECT x, y, AVG(temperature) as avg_temp
FROM df
WHERE temperature > 30
GROUP BY x, y
"""
result = duckdb.query(query).to_df()
print(result)


   x  y  temperature  humidity
0  0  0           30      0.20
1  1  1           35      0.25
2  2  2           40      0.30
   x  y  avg_temp
0  2  2      40.0
1  1  1      35.0
